In [1]:
import pandas as pd

from tqdm import tqdm 
from glob import glob
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.measure import label, regionprops_table
from skimage.transform import resize

from commons import binarize_image

In [2]:
def extract_props_from_image(mask):
    
    label_image = label(mask)

    feature_names = ['area',
                     'bbox',
                     'bbox_area',
                     'centroid', 
                     'convex_area', 
                     'eccentricity',
                     'equivalent_diameter', 
                     'euler_number',
                     'extent',
                     'filled_area',
                     'label',
                     'major_axis_length',
                     'minor_axis_length',
                     'moments',
                     'moments_central',
                     'orientation',
                     'perimeter',
                     'feret_diameter_max',
                     'solidity']
    
    return pd.DataFrame(regionprops_table(label_image, properties=feature_names))

In [3]:
out = pd.DataFrame()

path = "data/coletas/combined/masks/*"

for index in tqdm(glob(path)):
    
    try:
        
        image = imread(index)
        image_resized = resize(image, (224, 224), anti_aliasing=True)
        mask = binarize_image(image_resized)
        props = extract_props_from_image(mask)
        
        props['label'] = index.split('/')[-1]

        out = pd.concat([out, props], ignore_index=True)
    except Exception as e:
        print("Erro na imagem:", index, e)

100%|██████████████████████████████| 100/100 [01:08<00:00,  1.46it/s]


In [4]:
out.to_csv('features.csv', index=None)